<a href="https://colab.research.google.com/github/shreya-0806/mediccal-report-into-layman-language/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install gradio --upgrade
#!pip install faiss-cpu sentence-transformers transformers pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import pipeline
import textwrap



In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
summarizer = pipeline("summarization", model="google/pegasus-xsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def process_pdf_and_answer(file_obj, question):

    pdf_text = ""
    with pdfplumber.open(file_obj.name) as pdf:
        for page in pdf.pages:
            pdf_text += page.extract_text() + "\n"


    chunks = textwrap.wrap(pdf_text, width=500)
    embeddings = model.encode(chunks)
    dimension = embeddings[0].shape[0]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))


    def get_structured_answer(query):
        context = retrieve_context(query, index, chunks)
        prompt = f"Explain the following medical text in simple terms for a patient:\n\n{context}"
        result = summarizer(prompt, max_length=150, min_length=30, do_sample=False)
        return result[0]['summary_text']

    answers = {
        "Disease/Condition": get_structured_answer("What disease or condition is mentioned?"),
        "Symptoms": get_structured_answer("What symptoms are described?"),
        "Medications": get_structured_answer("What trestment plan are prescribed or recommended?")
    }


    formatted_output = "\n\n".join(
        f"🔹 **{category}**:\n{answer}"
        for category, answer in answers.items()
    )

    return formatted_output

def retrieve_context(query, index, chunks, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), top_k)
    return " ".join([chunks[i] for i in I[0]])

In [ ]:
# Gradio Interface with tabs
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🏥 Medical Report Explorer")
    gr.Markdown("Upload your medical report PDF and get easy-to-understand explanations")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Upload Medical Report (PDF)")
            question_input = gr.Textbox(label="Specific Question (Optional)", placeholder="Ask something specific about your report...")
            submit_btn = gr.Button("Analyze Report", variant="primary")

        with gr.Column():
            output = gr.Markdown(label="Your Report Summary")

    # Pre-defined questions
    with gr.Accordion("💡 Common Questions", open=False):
        gr.Examples(
            examples=[
                ["What's my main diagnosis?"],
                ["What symptoms should I watch for?"],
                ["What medications am I prescribed?"],
                ["What lifestyle changes are recommended?"]
            ],
            inputs=question_input
        )

    submit_btn.click(
        fn=process_pdf_and_answer,
        inputs=[file_input, question_input],
        outputs=output
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0e9f4436b9a24e361.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
